In [1]:
import json
import os
import requests
import datetime

MILLSECONDS_13_9_START = 1683061200000  # 2023년 05월 03일 수요일
MILLSECONDS_13_10_START = 1684270800000 # 2023년 05월 17일 수요일
MILLSECONDS_13_11_START = 1685566800000 # 2023년 06월 01일 목요일

DIR = r'C:\Users\김성윤\Desktop\data_collector'
API_KEY = 'RGAPI-1a1d24de-0002-4894-85cc-6deaf6ec560e'

def read_json(filename):
    with open(rf'{DIR}\{filename}') as f:
        return json.load(f)
    

def when_does_it_belongs_to(matchid):
    match_info = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/{matchid}?api_key={API_KEY}').json()['info']
    gameCreation = match_info['gameCreation']
    
    if gameCreation < MILLSECONDS_13_10_START:
        gameCreation = 9
    elif gameCreation < MILLSECONDS_13_11_START:
        gameCreation = 10
    else:
        gameCreation = 11
    
    gameDuration = match_info['gameDuration']//60

    is_blue_win = 1 if match_info['teams'][0]['win'] == True else 0
    return [gameCreation for i in range(16)], [gameDuration for i in range(16)] , [is_blue_win for i in range(16)]

In [2]:
game_files = [file for file in os.listdir(DIR) if file.startswith('KR')]
print("게임 갯수 :", len(game_files))

게임 갯수 : 382082


In [3]:
 # 리스트의 각 원소가 순차적으로 누적한다.
def transfomList(original_list):
    original_list = original_list
    new_list = []

    running_sum = 0
    for value in original_list:
        running_sum += value
        new_list.append(running_sum)

    return new_list

# 두 리스트의 차이를 반환한다. ( 블루팀 - 레드팀 )
def subtractList(blue_list1, red_list2):

    result = [a - b for a, b in zip(blue_list1, red_list2)]

    return result

def getMatchData(matchId):
    data = read_json(f'{matchId}')
    #### 블루팀 participantId 1 ~ 5 // 레드팀 participantId 6 ~ 10
    BLUE_IDs = [1,2,3,4,5]
    RED_IDs = [6,7,8,9,10]
    #### 여기는 프레임 인덱스가 N 일때, 실제 그 프레임이 담는 시간대는 N-1 분 0초부터 N-1 분 59초까지임
    #### 따라서 5분부터 15분 0초까지의 정보는 프레임 6분부터 15까지이다.
    frame_10_to_25 = data['info']['frames'][11:27]
    
    is_blue_win = data['info']
    # (해당 시간대에서) 각 팀별 킬 수, 어시스트 수, 누적 대형 오브젝트 처치 수 
    BLUE_KILLS,BLUE_DEATHS,BLUE_ASSISTS,BLUE_OBJECTS,BLUE_GOLDS,BLUE_EXPERIENCES  = [],[],[],[],[],[]   # 참고로 여기서는 각 챔피언들의 어시스트 합
    RED_KILLS,RED_DEATHS,RED_ASSISTS,RED_OBJECTS,RED_GOLDS,RED_EXPERIENCES = [],[],[],[],[],[]

    # patch , game_total_duration , is_blue_win 
    PATCH , LENGTH , IS_BLUE_WIN = when_does_it_belongs_to(matchId.split('.')[0])

    for frame in frame_10_to_25:

        blue_kills,blue_deaths,blue_assists,blue_objects,blue_golds,blue_experiences  = 0,0,0,0,0,0
        red_kills,red_deaths,red_assists,red_objects,red_golds,red_experiences = 0,0,0,0,0,0


        for event in frame['events']:

            if event['type'] == "CHAMPION_KILL":            # KILL, DEATH, ASSIST
                if event['killerId'] in BLUE_IDs:
                    blue_kills += 1
                if event['killerId'] in RED_IDs:
                    red_kills += 1
                if event['victimId'] in BLUE_IDs:
                    blue_deaths += 1
                if event['victimId'] in RED_IDs:
                    red_deaths += 1
                if event.get("assistingParticipantIds") != None:
                    for assistId in event.get("assistingParticipantIds"):
                        if assistId in BLUE_IDs:
                            blue_assists += 1
                        if assistId in RED_IDs:
                            red_assists += 1

            if event['type'] == "ELITE_MONSTER_KILL":       # ELITE MONSTER KILL
                if event['killerTeamId'] == 100:
                    blue_objects += 1
                else:
                    red_objects += 1
        

        for index in frame['participantFrames']:
            if int(index)  <= 5:
                
                blue_golds += frame['participantFrames'][index]['totalGold']
                blue_experiences += frame['participantFrames'][str(index)]['xp']
            else:
                red_golds += frame['participantFrames'][index]['totalGold']
                red_experiences += frame['participantFrames'][index]['xp']

        

        BLUE_KILLS.append(blue_kills)
        BLUE_DEATHS.append(blue_deaths)
        BLUE_ASSISTS.append(blue_assists)
        BLUE_OBJECTS.append(blue_objects)
        BLUE_GOLDS.append(blue_golds)
        BLUE_EXPERIENCES.append(blue_experiences)

        # # 누적 횟수로 전환
        
        # BLUE_GOLDS = transfomList(BLUE_GOLDS)
        # BLUE_EXPERIENCES = transfomList(BLUE_EXPERIENCES)

        RED_KILLS.append(red_kills)
        RED_DEATHS.append(red_deaths)
        RED_ASSISTS.append(red_assists)
        RED_OBJECTS.append(red_objects)
        RED_GOLDS.append(red_golds)
        RED_EXPERIENCES.append(red_experiences)

        # # 누적 횟수로 전환
        
        # RED_GOLDS = transfomList(RED_GOLDS)
        # RED_EXPERIENCES = transfomList(RED_EXPERIENCES)

    BLUE_KILLS = transfomList(BLUE_KILLS)
    BLUE_DEATHS = transfomList(BLUE_DEATHS)
    BLUE_ASSISTS = transfomList(BLUE_ASSISTS)
    BLUE_OBJECTS = transfomList(BLUE_OBJECTS)

    RED_KILLS = transfomList(RED_KILLS)
    RED_DEATHS = transfomList(RED_DEATHS)
    RED_ASSISTS = transfomList(RED_ASSISTS)
    RED_OBJECTS = transfomList(RED_OBJECTS)
        # 팀 차이 계산

    '''DIFF_KILLS = subtractList(BLUE_KILLS,RED_KILLS)
        DIFF_DEATHS = subtractList(BLUE_DEATHS,RED_DEATHS)
        DIFF_ASSISTS = subtractList(BLUE_ASSISTS,RED_ASSISTS)
        DIFF_OBJECTS = subtractList(BLUE_OBJECTS,RED_OBJECTS)
        DIFF_GOLDS = subtractList(BLUE_GOLDS,RED_GOLDS)
        DIFF_EXPERIENCES = subtractList(BLUE_EXPERIENCES,RED_EXPERIENCES)'''

            #위 순서대로 리스트 병합
    
    allData = []
    allData.append([matchId for i in range(16)])
    allData.append(list(range(10,26)))
    allData.append(BLUE_KILLS) , allData.append(BLUE_DEATHS) , allData.append(BLUE_ASSISTS) , allData.append(BLUE_OBJECTS) , allData.append(BLUE_GOLDS) , allData.append(BLUE_EXPERIENCES)
    allData.append(RED_KILLS) , allData.append(RED_DEATHS) , allData.append(RED_ASSISTS) , allData.append(RED_OBJECTS) , allData.append(RED_GOLDS) , allData.append(RED_EXPERIENCES) , allData.append(PATCH), allData.append(LENGTH), allData.append(IS_BLUE_WIN)

    
    return allData

import csv
from tqdm import tqdm

def writeCSVfile(matchIds): #위에서 추출한 정보들을 csv 파일로 저장한다. 모든 matchIds들에 대해서 진행.
        
        features = ['matchID']

        features.append('해당시각')

        features.append('블루킬')

        features.append('블루데스')

        features.append('블루어시')
    
        features.append('블루오브젝')

        features.append('블루골드')

        features.append('블루경험치')

    
        features.append('레드킬')
    
        features.append('레드데스')
    
        features.append('레드어시')
    
        features.append('레드오브젝')

        features.append('레드골드')

        features.append('레드경험치')


        features.append('패치')

        features.append('게임길이')

        features.append('블루승리여부')

        f = open(rf'{DIR}\MASTER_TIMELINE_DATA.csv','a', newline='')
        wr = csv.writer(f)
        wr.writerow(features)
        
        for matchId in tqdm(matchIds):
            try:
                rows = getMatchData(matchId)
                cols = list(map(list, zip(*rows)))
                wr.writerows(cols)
            except:
                pass

        f.close()

        print('csv write complete')

In [4]:
writeCSVfile(game_files)

 43%|████▎     | 162936/382082 [16:27:46<20:42:10,  2.94it/s] 